# DFS-DPP-GCN(D2GCN) 
For:

Learning from the Dark: Boosting Graph Convolutional Neural Networks wit Diverse Negative Samples.

This code can be run directly on colab

# 0 Preparing

In [ ]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-geometric

!pip install dppy

import torch
import numpy as np 
import networkx as nx
import matplotlib.pyplot as plt

from dppy.finite_dpps import FiniteDPP
from dppy.utils import example_eval_L_linear


     |████████████████████████████████| 3.0 MB 5.2 MB/s 
     |████████████████████████████████| 1.6 MB 5.3 MB/s 
     |████████████████████████████████| 222 kB 5.5 MB/s 
     |████████████████████████████████| 376 kB 42.3 MB/s 
     |████████████████████████████████| 45 kB 2.4 MB/s 
     |████████████████████████████████| 83 kB 1.2 MB/s 


## 0.1 Functions

In [ ]:
from collections import defaultdict
import numpy as np
def load_item_pop(X_train):
    item_pop = list()
    dd = defaultdict(list)
    #edge: <node1> <node2>
    for edge in X_train:
        #       <node2>             <node1> 
        dd[int(edge[1])].append(int(edge[0]))
    return dd

### 0.1.1 Get DFS path for each node

In [ ]:
import time
from collections import defaultdict

#Get DFS path for each node
class Personalized():
    def __init__(self, nx_G, mask, walks_num):
        self.G = nx_G
        self.mask = mask
        self.walks_num = walks_num


    def dfs(self, start_node):
        stack=[]
        stack.append(start_node)
        seen=set()
        seen.add(start_node)
        walks = []
        mask_list = set(self.mask[start_node])
        while (len(stack)>0):
            vertex=stack.pop()
            nodes=self.G[vertex]
            for w in nodes:
                if w not in seen:
                    stack.append(w)
                    seen.add(w)
            # Skip first-order nearest neighbors
            if vertex in mask_list:
                pass
            else:
                walks.append(vertex)
            if len(walks) >= self.walks_num:
                break
        return walks
    
    def intermediate(self):
        i = 0 
        candidate = defaultdict(list)
        for node in self.G.nodes():
            walk = self.dfs(node)
            candidate[node].extend(walk)
        return candidate


def candidate_choose(nx_Graph, mask, walks_num):
    G = Personalized(nx_Graph, mask, walks_num)
    #defaultdict(<class 'list'>, {967: [1186, 1222, 1145, 1119, 1453, 1112, 1131, 1084, 1834, 1295]})
    candidates = G.intermediate()
    return candidates

### 0.1.2 Basic GCN MessagePassing Layer

In [ ]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
from torch_scatter import gather_csr, scatter
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')  # "Add" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3-5: Start propagating messages.
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j, edge_index, size):
        # x_j has shape [E, out_channels]
        # edge_index has shape [2, E]

        # Step 3: Normalize node features.
        row, col = edge_index
        deg = degree(row, size[0], dtype=x_j.dtype)  # [N, ]
        deg_inv_sqrt = deg.pow(-0.5)   # [N, ]
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        return norm.view(-1, 1) * x_j
            
    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels]

        # Step 5: Return new node embeddings.
        return aggr_out

# 1. Load datasets

In [ ]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.datasets import KarateClub

# dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())
dataset = Planetoid(root='data/Planetoid', name='Citeseer', transform=NormalizeFeatures())
# dataset = Planetoid(root='data/Planetoid', name='Pubmed')


print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Processing...
Done!

Dataset: Citeseer():
Number of graphs: 1
Number of features: 3703
Number of classes: 6

Data(edge_index=[2, 9104], test_mask=[3327], train_mask=[3327], val_mask=[3327], x=[3327, 3703], y=[3327])
Number of nodes: 3327
Number of edges: 9104
Average node degree: 2.74
Number of training nodes: 120
Training node label rate: 0.04
Contains isolated nodes: True
Contains self-loops: False
Is undirected: True


## 1.1 Calculating the maximum connected subgraph

In [ ]:
from torch_geometric.utils import to_networkx, from_networkx,add_self_loops,to_networkx
import networkx as nx

# transform data from torch_geometric to networkx
G = to_networkx(data, to_undirected=True)
# Get the index of all points of the max-connected subgraph
Subnode = max(nx.connected_components(G), key=len)
print(len(Subnode))
# Get the subgraph
SubGraph = G.subgraph(Subnode)
SubnodeList = list(Subnode)
print(SubnodeList)

PyTSubGraph = from_networkx(SubGraph)
SubGraph = to_networkx(PyTSubGraph, to_undirected=True)

PyTSubGraph.test_mask = data.test_mask[SubnodeList]
PyTSubGraph.train_mask = data.train_mask[SubnodeList]
PyTSubGraph.val_mask = data.val_mask[SubnodeList]
PyTSubGraph.x = data.x[SubnodeList]
PyTSubGraph.y = data.y[SubnodeList]
print(PyTSubGraph)

# Get number of nodes
num_nodes = PyTSubGraph.num_nodes
# Get edge_index
edge_index = PyTSubGraph.edge_index
edge_value = torch.ones(edge_index.size(1), device=edge_index.device)
# Transforming the adjacency matrix from a sparse transpose matrix to a dense matrix
#(Without add_slef_loop)
adjdense = torch.sparse.FloatTensor(edge_index, edge_value, torch.Size([num_nodes,num_nodes])).to_dense()
print(adjdense.shape)

2120
[1, 5, 8, 10, 12, 13, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 38, 39, 40, 42, 43, 44, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 61, 62, 65, 66, 69, 70, 72, 75, 76, 77, 78, 79, 80, 81, 83, 84, 87, 88, 90, 91, 92, 93, 95, 96, 98, 99, 100, 101, 103, 104, 105, 106, 107, 110, 113, 114, 115, 118, 119, 122, 123, 124, 126, 128, 130, 131, 132, 134, 135, 136, 137, 138, 142, 144, 147, 148, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 162, 167, 168, 169, 170, 172, 173, 177, 178, 180, 181, 184, 186, 188, 189, 190, 191, 194, 195, 197, 198, 200, 201, 203, 204, 205, 206, 208, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 224, 226, 227, 228, 229, 230, 231, 232, 234, 236, 237, 240, 241, 242, 243, 244, 246, 247, 249, 250, 252, 253, 254, 255, 258, 259, 260, 263, 265, 266, 267, 268, 269, 272, 273, 274, 280, 285, 286, 287, 289, 292, 293, 294, 298, 300, 302, 303, 304, 307, 308, 311, 312, 313, 314, 316, 317, 318, 319, 321, 322, 325, 330, 331, 332, 333,

## 1.3 Generate graphs to DFS for each node

In [ ]:
#add self loop
Temp_edge_index, _ = add_self_loops(PyTSubGraph.edge_index, num_nodes=PyTSubGraph.x.size(0))
edge_index_list = torch.transpose(Temp_edge_index, 0, 1).tolist()

mask = load_item_pop(edge_index_list)

subG = to_networkx(PyTSubGraph, to_undirected=True)

#Length of DFS
walks_num = 5
candidates = candidate_choose(subG, mask, walks_num)
print(candidates)

defaultdict(<class 'list'>, {0: [1574, 1329, 69, 1178, 5], 1: [1712, 334, 1000, 1990, 1991], 2: [1829, 1244, 2018, 2074, 695], 3: [1840, 1857, 781, 1047, 1542], 4: [1584, 1583, 1542, 1986, 1872], 5: [1240, 1663, 1183, 1376, 1181], 6: [1690, 1715, 1714, 1697, 1716], 7: [383, 2060, 2059, 2058, 2024], 8: [1924, 1127, 911, 1415, 1554], 9: [1834, 902, 1840, 1857, 781], 10: [1991, 1984, 1392, 206, 1992], 11: [602, 1310, 2093, 1356, 123], 12: [178, 2118, 1830, 1900, 1927], 13: [1810, 1707, 1619, 1465, 1464], 14: [1536, 1961, 486, 2043, 2054], 15: [1774, 1882, 375, 2062, 1412], 16: [1815, 1727, 1328, 1832, 686], 17: [1733, 2098, 1730, 1732, 1230], 18: [1840, 1857, 781, 1047, 1536], 19: [1840, 1857, 781, 1047, 1536], 20: [1698, 1610, 1168, 1583, 1582], 21: [1930, 383, 2060, 2059, 2058], 22: [2054, 2046, 2045, 2044, 2014], 23: [1183, 1376, 1181, 1537, 49], 24: [1506, 900, 1468, 2096, 1983], 25: [1067, 1657, 2017, 1358, 1859], 26: [1715, 1714, 1697, 1716, 1838], 27: [1531, 1536, 1961, 486, 2043],

## 1.4 Take node 0 as an example，preview the the candidate set of DPPs
First-order nearest neighbor of nodes on a DFS path

In [ ]:
AdjList = []
for CentralIndex in candidates[0]:
    AjdIndex = torch.nonzero(adjdense[CentralIndex]) # get adj node index of central node
    AjdIndex = torch.reshape(AjdIndex, (-1,))
    AjdIndex = AjdIndex.tolist()
    AdjList.extend(AjdIndex)
    print(AjdIndex)
print(AdjList)

[623, 993, 1178, 1329, 1895]
[69, 1574]
[1329]
[5, 1574]
[743, 951, 1178, 1191, 1760]
[623, 993, 1178, 1329, 1895, 69, 1574, 1329, 5, 1574, 743, 951, 1178, 1191, 1760]


# 2 Model Definition
## 2.1 Defining the D2GCN model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
from dppy.finite_dpps import FiniteDPP
from dppy.utils import example_eval_L_linear
import time

#-------------
#NegRate can directly set as 1 or 2 
#-------------

class D2GCN(torch.nn.Module):
    def __init__(self, hidden_channels,candidates,adjdense,num_layers,num_nodes):
        super(D2GCN, self).__init__()
        # torch.manual_seed(12345)

        self.num_layers = num_layers
        self.CONVs = torch.nn.ModuleList()
        self.DPPCONVs = torch.nn.ModuleList()
        self.NegRates = nn.ParameterList()
        for layer in range(self.num_layers-1):
            if layer == 0:
                self.CONVs.append(GCNConv(dataset.num_features, hidden_channels))
                self.DPPCONVs.append(GCNConv(dataset.num_features, hidden_channels))
                self.NegRates.append(nn.Parameter(torch.FloatTensor([1])))
            else:
                self.CONVs.append(GCNConv(hidden_channels, hidden_channels))
                self.DPPCONVs.append(GCNConv(hidden_channels, hidden_channels))
                self.NegRates.append(nn.Parameter(torch.FloatTensor([1])))
        self.CONVs.append(GCNConv(hidden_channels, dataset.num_classes))
        self.DPPCONVs.append(GCNConv(hidden_channels, dataset.num_classes))
        self.NegRates.append(nn.Parameter(torch.FloatTensor([1])))

        self.candidates = candidates
        self.adjdense = adjdense
        self.num_nodes = num_nodes
        # According to experimental experience, setting NegRate to 1~3 can achieve better results
        self.NegRate = 1


    def forward(self, x, edge_index,DPPedge_index_list,train_neg_rate):
        for layer in range(self.num_layers):
          x_temp = x.clone()
          posi_x = self.CONVs[layer](x, edge_index)
          if len(DPPedge_index_list) < self.num_layers:
            DppMatrix = self.GetDppMatrix(x_temp)
            #change dense back to sparse
            Dppedge_index = torch.nonzero(DppMatrix).t()
            DPPedge_index_list.append(Dppedge_index)
          else:
            Dppedge_index = DPPedge_index_list[layer]

          nega_x = self.DPPCONVs[layer](x_temp, Dppedge_index)
          if train_neg_rate: #NegRate is a trainable parameter
            x = posi_x - self.NegRates[layer] * nega_x
          else:
            x = posi_x - self.NegRate * nega_x

          if layer < (self.num_layers-1):
            x = x.relu()
            x = F.dropout(x, p=0.5, training=self.training)
        return x,DPPedge_index_list
    
    def GetDppMatrix(self,x):
        DppMatrix = torch.zeros((self.num_nodes,self.num_nodes))
        for node in self.candidates:
          AdjList = []
          for CentralIndex in self.candidates[node]:
              AjdIndex = torch.nonzero(self.adjdense[CentralIndex]) # get adj node index of central node
              AjdIndex = torch.reshape(AjdIndex, (-1,))
              AjdIndex = AjdIndex.tolist()
              AdjList.extend(AjdIndex)
          sampleAmount = int(adjdense[node].sum().item()) + 1
          if sampleAmount > len(AdjList):
            sampleAmount = len(AdjList)
          DppIndex = self.Dppsample(x, AdjList, sampleAmount)
          DppMatrix[node][DppIndex] = 1
        return DppMatrix
    
    def Dppsample(self,x, AjdIndex,sampleAmount):
        Index = self.Rek_dpp_kernel(x[AjdIndex], sampleAmount)
        AjdIndex = np.array(AjdIndex)
        DppIndex = AjdIndex[Index]
        return DppIndex

    def Rek_dpp_kernel(self, weight, k_number):
        normTemp = weight.norm(dim=1)[:, None]
        oneTemp = torch.ones_like(normTemp)
        normTemp = torch.where(normTemp == 0,oneTemp,normTemp)
        weight_norm = weight / normTemp
        cosSim = torch.mm(weight_norm, weight_norm.transpose(0,1)).detach().numpy()
        Lmatrix = np.exp(-(1-cosSim))
        Lmatrix = Lmatrix + 0.01*np.eye(Lmatrix.shape[0])
        Lmatrix = np.float64(Lmatrix)
        DPP = FiniteDPP('likelihood', **{'L': Lmatrix})
        
        DPP.flush_samples()
        DPP.sample_exact_k_dpp(size=k_number)
        values = np.squeeze(DPP.list_of_samples)
        
        return values

## 2.2 Defining the MAD

In [ ]:
def MAD(x):
  x_norm = x / x.norm(dim=1)[:, None]
  dist = 1-torch.mm(x_norm, x_norm.transpose(0,1))
  one1 = torch.ones_like(dist)
  zreo1 = torch.zeros_like(dist)
  dist_1 = torch.where(dist > 0, one1, zreo1)
  D = dist.sum(0)/dist_1.sum(0)
  one = torch.ones_like(D)
  zero = torch.zeros_like(D)
  D_1 = torch.where(D > 0, one, zero)
  Mad = D.sum()/D_1.sum()
  return Mad.detach().numpy()


#3 Training

In [ ]:
import sys

def test(DPPedge_index_list,train_neg_rate):
      model.eval()
      out,_ = model(PyTSubGraph.x, edge_index,DPPedge_index_list,train_neg_rate)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[PyTSubGraph.test_mask] == PyTSubGraph.y[PyTSubGraph.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(PyTSubGraph.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc

#----------------
# Set training parameters
num_layers = 5
Runs = 11
Epochs = 201
#--------------------
#Set whether NegRate is a trainable parameter
#if False:
#   NegRate = 1
train_neg_rate = True
#-----------------

for num in range(1,Runs):
  model = D2GCN(hidden_channels=16,
          candidates=candidates,
          adjdense=adjdense,
          num_layers=num_layers,
          num_nodes=num_nodes)
  if num == 1 :
    print(model)
  print("num_layers",num_layers)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
  criterion = torch.nn.CrossEntropyLoss()

  for epoch in range(1, Epochs):
      start = time.time()
      DPPedge_index_list = []
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out,DPPedge_index_list = model(PyTSubGraph.x, edge_index,DPPedge_index_list,train_neg_rate)  
      loss = criterion(out[PyTSubGraph.train_mask], PyTSubGraph.y[PyTSubGraph.train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()
      end = time.time()
      if epoch == 1:
          print("---------------")
          print("DPPedge_index_list",DPPedge_index_list[0].shape)
          print("Epoch time:",end-start)
          print("---------------")
      if epoch % 10 == 0 or epoch >190:
          test_acc = test(DPPedge_index_list,train_neg_rate)
          print(f'num: {num:03d}, Epoch: {epoch:03d}, Loss: {loss:.4f},Test Accuracy: {test_acc:.4f},MAD: {MAD(out):.4f}')

D2GCN(
  (CONVs): ModuleList(
    (0): GCNConv(
      (lin): Linear(in_features=3703, out_features=16, bias=True)
    )
    (1): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (2): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (3): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (4): GCNConv(
      (lin): Linear(in_features=16, out_features=6, bias=True)
    )
  )
  (DPPCONVs): ModuleList(
    (0): GCNConv(
      (lin): Linear(in_features=3703, out_features=16, bias=True)
    )
    (1): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (2): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (3): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (4): GCNConv(
      (lin): Linear(in_features=16, out_features=6, bias=True)
    )
  )
  (NegRates): ParameterList(
      (0): Parameter cont